In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 12s 0us/step


In [3]:
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

x_train = x_train / 255.0
x_test = x_test / 255.0

In [4]:
# 裝回tf.data.Dataset

ds_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
ds_train

<TensorSliceDataset shapes: ((32, 32, 3), (10,)), types: (tf.float64, tf.float32)>

In [5]:
# 取得單一batch

one_batch = ds_train.batch(batch_size=128).take(1)

for (x, y) in one_batch:
    print(x.shape)
    print(y.shape)

(128, 32, 32, 3)
(128, 10)


In [6]:
# 透過模型去擬合


model = keras.Sequential([
    layers.InputLayer(input_shape=(32, 32, 3)),
    layers.Conv2D(64, 3, padding='same'),
    layers.MaxPooling2D(),
    layers.GlobalMaxPooling2D(),
    layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics='acc'
)

model.fit(one_batch.repeat(),epochs=20, steps_per_epoch=100, validation_data=(x_test, y_test))

Epoch 1/20
100/100 [==============================] - 31s 28ms/step - loss: 2.1611 - acc: 0.2389 - val_loss: 2.2715 - val_acc: 0.1643
Epoch 2/20
100/100 [==============================] - 2s 24ms/step - loss: 1.8628 - acc: 0.3634 - val_loss: 2.1789 - val_acc: 0.2044
Epoch 3/20
100/100 [==============================] - 2s 24ms/step - loss: 1.4618 - acc: 0.5195 - val_loss: 2.2548 - val_acc: 0.2165
Epoch 4/20
100/100 [==============================] - 2s 24ms/step - loss: 1.0719 - acc: 0.7159 - val_loss: 2.4398 - val_acc: 0.2194
Epoch 5/20
100/100 [==============================] - 2s 24ms/step - loss: 0.7487 - acc: 0.8519 - val_loss: 2.6736 - val_acc: 0.2240
Epoch 6/20
100/100 [==============================] - 2s 24ms/step - loss: 0.5087 - acc: 0.9497 - val_loss: 2.9279 - val_acc: 0.2254
Epoch 7/20
100/100 [==============================] - 2s 24ms/step - loss: 0.3414 - acc: 0.9835 - val_loss: 3.1942 - val_acc: 0.2281
Epoch 8/20
100/100 [==============================] - 2s 24ms/step -

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
global_max_pooling2d (Global (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 10)                650       
Total params: 2,442
Trainable params: 2,442
Non-trainable params: 0
_________________________________________________________________


In [12]:
# 上面是的確適合圖片的神經網路，那麼讓我們看看另外一個神經網路，單純用MLP去做

model_mlp = keras.Sequential([
    layers.InputLayer(input_shape=(32, 32, 3)),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model_mlp.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['acc']
)
model_mlp.fit(
    x=one_batch.repeat(),
    epochs=20,
    steps_per_epoch=100,
    validation_data=(x_test, y_test)
)

Epoch 1/20
100/100 [==============================] - 3s 21ms/step - loss: 1.4518 - acc: 0.5409 - val_loss: 2.5513 - val_acc: 0.2125
Epoch 2/20
100/100 [==============================] - 1s 13ms/step - loss: 0.4192 - acc: 0.9570 - val_loss: 3.2334 - val_acc: 0.2162
Epoch 3/20
100/100 [==============================] - 1s 12ms/step - loss: 0.1080 - acc: 1.0000 - val_loss: 3.8863 - val_acc: 0.2147
Epoch 4/20
100/100 [==============================] - 1s 13ms/step - loss: 0.0414 - acc: 1.0000 - val_loss: 4.3444 - val_acc: 0.2158
Epoch 5/20
100/100 [==============================] - 1s 13ms/step - loss: 0.0220 - acc: 1.0000 - val_loss: 4.6720 - val_acc: 0.2145
Epoch 6/20
100/100 [==============================] - 1s 12ms/step - loss: 0.0137 - acc: 1.0000 - val_loss: 4.9287 - val_acc: 0.2143
Epoch 7/20
100/100 [==============================] - 1s 13ms/step - loss: 0.0095 - acc: 1.0000 - val_loss: 5.1437 - val_acc: 0.2144
Epoch 8/20
100/100 [==============================] - 1s 12ms/step - 

In [13]:
model_mlp.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 32)                98336     
_________________________________________________________________
dense_8 (Dense)              (None, 10)                330       
Total params: 98,666
Trainable params: 98,666
Non-trainable params: 0
_________________________________________________________________


> 透過single_batch去測試模型fit能力，雖然模型不大，但fit能力足夠，但很顯然general是差的。

In [15]:
# 接著，去fit整個dataset

model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
391/391 [==============================] - 5s 12ms/step - loss: 2.2186 - acc: 0.3148 - val_loss: 2.1223 - val_acc: 0.3189
Epoch 2/10
391/391 [==============================] - 4s 11ms/step - loss: 2.0652 - acc: 0.3299 - val_loss: 2.0113 - val_acc: 0.3388
Epoch 3/10
391/391 [==============================] - 4s 11ms/step - loss: 1.9690 - acc: 0.3429 - val_loss: 1.9278 - val_acc: 0.3555
Epoch 4/10
391/391 [==============================] - 4s 11ms/step - loss: 1.9119 - acc: 0.3530 - val_loss: 1.8722 - val_acc: 0.3636
Epoch 5/10
391/391 [==============================] - 5s 12ms/step - loss: 1.8627 - acc: 0.3625 - val_loss: 1.8305 - val_acc: 0.3742
Epoch 6/10
391/391 [==============================] - 4s 11ms/step - loss: 1.8249 - acc: 0.3702 - val_loss: 1.8182 - val_acc: 0.3783
Epoch 7/10
391/391 [==============================] - 4s 10ms/step - loss: 1.7935 - acc: 0.3780 - val_loss: 1.7691 - val_acc: 0.3853
Epoch 8/10
391/391 [==============================] - 5s 12ms/step - 

In [16]:
model_mlp.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
391/391 [==============================] - 2s 5ms/step - loss: 2.4048 - acc: 0.1007 - val_loss: 2.3028 - val_acc: 0.1000
Epoch 2/10
391/391 [==============================] - 2s 5ms/step - loss: 2.2946 - acc: 0.1067 - val_loss: 2.2797 - val_acc: 0.1323
Epoch 3/10
391/391 [==============================] - 2s 5ms/step - loss: 2.2388 - acc: 0.1478 - val_loss: 2.1795 - val_acc: 0.1716
Epoch 4/10
391/391 [==============================] - 2s 5ms/step - loss: 2.1388 - acc: 0.1754 - val_loss: 2.1051 - val_acc: 0.1765
Epoch 5/10
391/391 [==============================] - 2s 5ms/step - loss: 2.0929 - acc: 0.1754 - val_loss: 2.0801 - val_acc: 0.1754
Epoch 6/10
391/391 [==============================] - 2s 5ms/step - loss: 2.0709 - acc: 0.1812 - val_loss: 2.0617 - val_acc: 0.1797
Epoch 7/10
391/391 [==============================] - 2s 5ms/step - loss: 2.0592 - acc: 0.1786 - val_loss: 2.0533 - val_acc: 0.1836
Epoch 8/10
391/391 [==============================] - 2s 5ms/step - loss: 2.

> 這邊用全部資料可以發現，CNN模型雖然參數小，但是用正確的方式學習到參數，雖然只跑了10個epoch不足以達到適用的效果，但整體趨勢樂觀。而MLP則是效果不好，但也是有學習的趨勢。

> 那我們再試看看，多一點epoch會如何(因為兩個模型可以看到都在學習中)


In [17]:
model.fit(x_train, y_train, batch_size=128, epochs=90, validation_data=(x_test, y_test))

Epoch 1/90
391/391 [==============================] - 4s 11ms/step - loss: 1.7074 - acc: 0.3995 - val_loss: 1.7087 - val_acc: 0.3979
Epoch 2/90
391/391 [==============================] - 4s 9ms/step - loss: 1.6940 - acc: 0.4029 - val_loss: 1.6824 - val_acc: 0.4105
Epoch 3/90
391/391 [==============================] - 4s 9ms/step - loss: 1.6771 - acc: 0.4091 - val_loss: 1.6714 - val_acc: 0.4155
Epoch 4/90
391/391 [==============================] - 4s 9ms/step - loss: 1.6634 - acc: 0.4126 - val_loss: 1.6684 - val_acc: 0.4133
Epoch 5/90
391/391 [==============================] - 4s 9ms/step - loss: 1.6526 - acc: 0.4163 - val_loss: 1.6500 - val_acc: 0.4200
Epoch 6/90
391/391 [==============================] - 4s 9ms/step - loss: 1.6415 - acc: 0.4195 - val_loss: 1.6378 - val_acc: 0.4225
Epoch 7/90
391/391 [==============================] - 4s 9ms/step - loss: 1.6321 - acc: 0.4233 - val_loss: 1.6447 - val_acc: 0.4230
Epoch 8/90
391/391 [==============================] - 4s 10ms/step - loss: 

In [18]:
model_mlp.fit(x_train, y_train, batch_size=128, epochs=90, validation_data=(x_test, y_test))

Epoch 1/90
391/391 [==============================] - 2s 5ms/step - loss: 2.0299 - acc: 0.1989 - val_loss: 2.0203 - val_acc: 0.2019
Epoch 2/90
391/391 [==============================] - 2s 5ms/step - loss: 2.0050 - acc: 0.2146 - val_loss: 1.9867 - val_acc: 0.2236
Epoch 3/90
391/391 [==============================] - 2s 5ms/step - loss: 1.9727 - acc: 0.2417 - val_loss: 1.9450 - val_acc: 0.2535
Epoch 4/90
391/391 [==============================] - 2s 5ms/step - loss: 1.9431 - acc: 0.2545 - val_loss: 1.9359 - val_acc: 0.2595
Epoch 5/90
391/391 [==============================] - 2s 5ms/step - loss: 1.9352 - acc: 0.2584 - val_loss: 1.9300 - val_acc: 0.2609
Epoch 6/90
391/391 [==============================] - 2s 5ms/step - loss: 1.9279 - acc: 0.2586 - val_loss: 1.9246 - val_acc: 0.2601
Epoch 7/90
391/391 [==============================] - 2s 5ms/step - loss: 1.9249 - acc: 0.2571 - val_loss: 1.9235 - val_acc: 0.2586
Epoch 8/90
391/391 [==============================] - 2s 5ms/step - loss: 1.

> 可以發現MLP似乎到達了一個瓶頸，兩個模型其實都有些underfitting，那麼我們選擇表現較好的CNN類型去做擴充

In [20]:
model_cnn_multi = keras.Sequential([
    layers.InputLayer(input_shape=(32, 32, 3)),
    layers.Conv2D(64, 3, padding='same'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, padding='same'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, padding='same'),
    layers.MaxPooling2D(),
    layers.GlobalMaxPooling2D(),
    layers.Dense(10, activation='softmax')
])


model_cnn_multi.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['acc']
)

model_cnn_multi.fit(
    x=x_train,
    y=y_train,
    epochs=100,
    batch_size=128,
    validation_data=(x_test, y_test)
)

Epoch 1/100
391/391 [==============================] - 9s 21ms/step - loss: 1.4443 - acc: 0.4812 - val_loss: 1.1029 - val_acc: 0.6073
Epoch 2/100
391/391 [==============================] - 8s 19ms/step - loss: 1.0146 - acc: 0.6440 - val_loss: 0.9811 - val_acc: 0.6587
Epoch 3/100
391/391 [==============================] - 8s 19ms/step - loss: 0.8223 - acc: 0.7161 - val_loss: 0.8745 - val_acc: 0.6993
Epoch 4/100
391/391 [==============================] - 8s 20ms/step - loss: 0.6845 - acc: 0.7649 - val_loss: 0.8140 - val_acc: 0.7208
Epoch 5/100
391/391 [==============================] - 8s 20ms/step - loss: 0.5691 - acc: 0.8021 - val_loss: 0.8027 - val_acc: 0.7243
Epoch 6/100
391/391 [==============================] - 8s 19ms/step - loss: 0.4776 - acc: 0.8333 - val_loss: 0.8597 - val_acc: 0.7250
Epoch 7/100
391/391 [==============================] - 8s 20ms/step - loss: 0.3814 - acc: 0.8667 - val_loss: 0.8916 - val_acc: 0.7289
Epoch 8/100
391/391 [==============================] - 8s 19ms

KeyboardInterrupt: ignored

> 可以看到模型學習非常快，效果也不錯！但是馬上就overfitting了，這邊需要加強泛化能力

> 發現沒有用activation 先用一下!

- BatchNormalization
    - 激活函數之前
- Dropout

In [22]:
model_cnn_multi_av = keras.Sequential([
    layers.InputLayer(input_shape=(32, 32, 3)),
    layers.Conv2D(64, 3, padding='same'),
    layers.Activation('relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same'),
    layers.Activation('relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same'),
    layers.Activation('relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, padding='same'),
    layers.Activation('relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, padding='same'),
    layers.Activation('relu'),
    layers.MaxPooling2D(),
    layers.GlobalMaxPooling2D(),
    layers.Dense(10, activation='softmax')
])


model_cnn_multi_av.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['acc']
)

model_cnn_multi_av.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=128,
    validation_data=(x_test, y_test)
)

Epoch 1/30
391/391 [==============================] - 9s 21ms/step - loss: 1.5961 - acc: 0.4106 - val_loss: 1.3136 - val_acc: 0.5289
Epoch 2/30
391/391 [==============================] - 8s 19ms/step - loss: 1.1705 - acc: 0.5785 - val_loss: 1.0379 - val_acc: 0.6357
Epoch 3/30
391/391 [==============================] - 8s 20ms/step - loss: 0.9750 - acc: 0.6578 - val_loss: 1.0355 - val_acc: 0.6366
Epoch 4/30
391/391 [==============================] - 8s 20ms/step - loss: 0.8616 - acc: 0.6973 - val_loss: 0.8995 - val_acc: 0.6814
Epoch 5/30
391/391 [==============================] - 8s 20ms/step - loss: 0.7597 - acc: 0.7343 - val_loss: 0.8419 - val_acc: 0.7046
Epoch 6/30
391/391 [==============================] - 8s 20ms/step - loss: 0.6854 - acc: 0.7605 - val_loss: 0.7990 - val_acc: 0.7273
Epoch 7/30
391/391 [==============================] - 8s 20ms/step - loss: 0.6179 - acc: 0.7824 - val_loss: 0.8642 - val_acc: 0.7143
Epoch 8/30
391/391 [==============================] - 8s 20ms/step - 

> 有activation function透過非線性的擬合能力，幫助模型效果更好了一些!

In [23]:
model_cnn_multi_av_bn = keras.Sequential([
    layers.InputLayer(input_shape=(32, 32, 3)),
    layers.Conv2D(64, 3, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D(),
    layers.GlobalMaxPooling2D(),
    layers.Dense(10, activation='softmax')
])


model_cnn_multi_av_bn.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['acc']
)

model_cnn_multi_av_bn.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=128,
    validation_data=(x_test, y_test)
)

Epoch 1/30
391/391 [==============================] - 11s 26ms/step - loss: 1.2068 - acc: 0.5682 - val_loss: 1.6515 - val_acc: 0.4216
Epoch 2/30
391/391 [==============================] - 10s 25ms/step - loss: 0.8222 - acc: 0.7115 - val_loss: 1.3895 - val_acc: 0.5515
Epoch 3/30
391/391 [==============================] - 10s 25ms/step - loss: 0.6715 - acc: 0.7649 - val_loss: 0.8497 - val_acc: 0.7076
Epoch 4/30
391/391 [==============================] - 10s 25ms/step - loss: 0.5778 - acc: 0.7990 - val_loss: 1.0218 - val_acc: 0.6727
Epoch 5/30
391/391 [==============================] - 10s 25ms/step - loss: 0.4928 - acc: 0.8275 - val_loss: 0.8761 - val_acc: 0.7135
Epoch 6/30
391/391 [==============================] - 10s 25ms/step - loss: 0.4182 - acc: 0.8556 - val_loss: 1.1508 - val_acc: 0.6502
Epoch 7/30
391/391 [==============================] - 10s 25ms/step - loss: 0.3634 - acc: 0.8726 - val_loss: 1.2062 - val_acc: 0.6669
Epoch 8/30
391/391 [==============================] - 10s 25ms

> 加入BN之後，效果: 可以看到generalize有些許提升，但並不穩定。

> [試試看Dropout](https://kknews.cc/zh-tw/code/mnvee3p.html) --> CNN Dropout 意義不大，因為神經元本來就很少!


In [25]:
model_cnn_multi_av_drop = keras.Sequential([
    layers.InputLayer(input_shape=(32, 32, 3)),
    layers.Conv2D(64, 3, padding='same'),
    layers.Dropout(0.5),
    layers.Activation('relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same'),
    layers.Dropout(0.5),
    layers.Activation('relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same'),
    layers.Dropout(0.5),
    layers.Activation('relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, padding='same'),
    layers.Dropout(0.5),
    layers.Activation('relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, padding='same'),
    layers.Dropout(0.5),
    layers.Activation('relu'),
    layers.MaxPooling2D(),
    layers.GlobalMaxPooling2D(),
    layers.Dense(10, activation='softmax')
])


model_cnn_multi_av_drop.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['acc']
)

model_cnn_multi_av_drop.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=128,
    validation_data=(x_test, y_test)
)

Epoch 1/30
391/391 [==============================] - 12s 28ms/step - loss: 1.7398 - acc: 0.3495 - val_loss: 2.0828 - val_acc: 0.2929
Epoch 2/30
391/391 [==============================] - 10s 26ms/step - loss: 1.4094 - acc: 0.4841 - val_loss: 1.9627 - val_acc: 0.3518
Epoch 3/30
391/391 [==============================] - 10s 26ms/step - loss: 1.2644 - acc: 0.5430 - val_loss: 1.8666 - val_acc: 0.4210
Epoch 4/30
391/391 [==============================] - 10s 26ms/step - loss: 1.1783 - acc: 0.5779 - val_loss: 1.8704 - val_acc: 0.3473
Epoch 5/30
391/391 [==============================] - 10s 26ms/step - loss: 1.1108 - acc: 0.6016 - val_loss: 1.7396 - val_acc: 0.4727
Epoch 6/30
391/391 [==============================] - 10s 26ms/step - loss: 1.0594 - acc: 0.6198 - val_loss: 1.7191 - val_acc: 0.4521
Epoch 7/30
391/391 [==============================] - 10s 26ms/step - loss: 1.0122 - acc: 0.6407 - val_loss: 1.6688 - val_acc: 0.5048
Epoch 8/30
391/391 [==============================] - 10s 26ms

> 可以看到，的確反而讓模型更難以學習!